# Visitor Pattern

## Intent
Represent an operation to be performed on elements of an object structure. Visitor lets you define a new operation without changing the classes of the elements on which it operates.

## Problem
You have a complex object structure and:
- Need to perform different operations on objects
- Operations change frequently
- Don't want to pollute classes with unrelated operations
- Want to separate algorithms from objects

**Real-world analogy**: Insurance agent visiting different types of buildings - same visitor, different operations per building type

## When to Use
✅ **Use when:**
- Object structure rarely changes
- Need to perform many unrelated operations
- Operations change frequently
- Want to keep related operations together
- Need to accumulate state while traversing structure

❌ **Avoid when:**
- Object structure changes frequently (breaks all visitors)
- Few operations on objects
- Operations are closely related to objects

## Pattern Structure
```
┌────────┐         ┌─────────┐
│Visitor │         │ Element │
├────────┤         ├─────────┤
│visitA()│         │accept() │
│visitB()│         └─────────┘
└────────┘              ▲
    ▲              ┌────┴────┐
    │         ┌────┴───┐┌────┴───┐
┌───┴────┐   │Element ││Element │
│Concrete│   │   A    ││   B    │
│Visitor │   └────────┘└────────┘
└────────┘
```

## Key Concept: Double Dispatch

The visitor pattern uses **double dispatch**:
1. Element calls `visitor.visit(this)`
2. Visitor calls element-specific method

This determines both the visitor type AND element type at runtime.

## Example 1: Without Visitor

**Problem**: Adding new operations requires modifying all classes

In [ ]:
# WITHOUT Visitor - Operations mixed with structure
class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    def draw(self):
        print(f"Drawing circle r={self.radius}")
    
    def calculate_area(self):
        return 3.14 * self.radius ** 2
    
    def export_xml(self):
        return f"<circle r='{self.radius}'/>"
    
    # Adding new operation requires modifying this class!
    # def export_json(self): ...

class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def draw(self):
        print(f"Drawing rectangle {self.width}x{self.height}")
    
    def calculate_area(self):
        return self.width * self.height
    
    def export_xml(self):
        return f"<rect w='{self.width}' h='{self.height}'/>"

print("\n❌ Operations mixed with shape classes!")
print("❌ Adding new operation modifies all classes!")
print("❌ Violates Open/Closed Principle!")

## Implementation: Visitor Pattern

In [ ]:
from abc import ABC, abstractmethod
from typing import List
import math

# Visitor interface
class ShapeVisitor(ABC):
    """Abstract visitor for shapes."""
    
    @abstractmethod
    def visit_circle(self, circle: 'Circle') -> None:
        pass
    
    @abstractmethod
    def visit_rectangle(self, rectangle: 'Rectangle') -> None:
        pass
    
    @abstractmethod
    def visit_triangle(self, triangle: 'Triangle') -> None:
        pass


# Element interface
class Shape(ABC):
    """Abstract shape."""
    
    @abstractmethod
    def accept(self, visitor: ShapeVisitor) -> None:
        """Accept visitor (double dispatch)."""
        pass


# Concrete elements
class Circle(Shape):
    def __init__(self, radius: float):
        self.radius = radius
    
    def accept(self, visitor: ShapeVisitor) -> None:
        visitor.visit_circle(self)


class Rectangle(Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height
    
    def accept(self, visitor: ShapeVisitor) -> None:
        visitor.visit_rectangle(self)


class Triangle(Shape):
    def __init__(self, base: float, height: float):
        self.base = base
        self.height = height
    
    def accept(self, visitor: ShapeVisitor) -> None:
        visitor.visit_triangle(self)


# Concrete visitors (operations)
class AreaCalculator(ShapeVisitor):
    """Calculates area of shapes."""
    
    def __init__(self):
        self.total_area = 0
    
    def visit_circle(self, circle: Circle) -> None:
        area = math.pi * circle.radius ** 2
        print(f"  📐 Circle (r={circle.radius}): area = {area:.2f}")
        self.total_area += area
    
    def visit_rectangle(self, rectangle: Rectangle) -> None:
        area = rectangle.width * rectangle.height
        print(f"  📐 Rectangle ({rectangle.width}x{rectangle.height}): area = {area:.2f}")
        self.total_area += area
    
    def visit_triangle(self, triangle: Triangle) -> None:
        area = 0.5 * triangle.base * triangle.height
        print(f"  📐 Triangle (b={triangle.base},h={triangle.height}): area = {area:.2f}")
        self.total_area += area


class XMLExporter(ShapeVisitor):
    """Exports shapes to XML."""
    
    def __init__(self):
        self.xml_parts = []
    
    def visit_circle(self, circle: Circle) -> None:
        self.xml_parts.append(f"<circle r='{circle.radius}'/>")
        print(f"  📄 Exported circle to XML")
    
    def visit_rectangle(self, rectangle: Rectangle) -> None:
        self.xml_parts.append(f"<rectangle w='{rectangle.width}' h='{rectangle.height}'/>")
        print(f"  📄 Exported rectangle to XML")
    
    def visit_triangle(self, triangle: Triangle) -> None:
        self.xml_parts.append(f"<triangle b='{triangle.base}' h='{triangle.height}'/>")
        print(f"  📄 Exported triangle to XML")
    
    def get_xml(self) -> str:
        return "<shapes>\n  " + "\n  ".join(self.xml_parts) + "\n</shapes>"


class DrawVisitor(ShapeVisitor):
    """Draws shapes."""
    
    def visit_circle(self, circle: Circle) -> None:
        print(f"  🎨 Drawing circle with radius {circle.radius}")
    
    def visit_rectangle(self, rectangle: Rectangle) -> None:
        print(f"  🎨 Drawing rectangle {rectangle.width}x{rectangle.height}")
    
    def visit_triangle(self, triangle: Triangle) -> None:
        print(f"  🎨 Drawing triangle base={triangle.base} height={triangle.height}")


# Demo
print("\n=== Visitor Pattern ===")

# Create shapes
shapes: List[Shape] = [
    Circle(5),
    Rectangle(4, 6),
    Triangle(3, 8),
    Circle(3)
]

print("\n1. Calculate areas:")
area_calc = AreaCalculator()
for shape in shapes:
    shape.accept(area_calc)
print(f"  📊 Total area: {area_calc.total_area:.2f}")

print("\n2. Export to XML:")
xml_exporter = XMLExporter()
for shape in shapes:
    shape.accept(xml_exporter)
print(f"\n{xml_exporter.get_xml()}")

print("\n3. Draw shapes:")
drawer = DrawVisitor()
for shape in shapes:
    shape.accept(drawer)

print("\n✅ Added 3 operations without modifying shape classes!")

## Real-World Example: File System Operations

In [ ]:
# Visitor
class FileSystemVisitor(ABC):
    @abstractmethod
    def visit_file(self, file: 'File') -> None:
        pass
    
    @abstractmethod
    def visit_directory(self, directory: 'Directory') -> None:
        pass


# Elements
class FileSystemElement(ABC):
    def __init__(self, name: str):
        self.name = name
    
    @abstractmethod
    def accept(self, visitor: FileSystemVisitor) -> None:
        pass


class File(FileSystemElement):
    def __init__(self, name: str, size: int):
        super().__init__(name)
        self.size = size
    
    def accept(self, visitor: FileSystemVisitor) -> None:
        visitor.visit_file(self)


class Directory(FileSystemElement):
    def __init__(self, name: str):
        super().__init__(name)
        self.children: List[FileSystemElement] = []
    
    def add(self, element: FileSystemElement) -> None:
        self.children.append(element)
    
    def accept(self, visitor: FileSystemVisitor) -> None:
        visitor.visit_directory(self)
        # Visit children
        for child in self.children:
            child.accept(visitor)


# Concrete visitors
class SizeCalculator(FileSystemVisitor):
    """Calculate total size."""
    
    def __init__(self):
        self.total_size = 0
    
    def visit_file(self, file: File) -> None:
        self.total_size += file.size
        print(f"  📄 {file.name}: {file.size} bytes")
    
    def visit_directory(self, directory: Directory) -> None:
        print(f"  📁 {directory.name}/")


class FileSearcher(FileSystemVisitor):
    """Search for files by extension."""
    
    def __init__(self, extension: str):
        self.extension = extension
        self.found = []
    
    def visit_file(self, file: File) -> None:
        if file.name.endswith(self.extension):
            self.found.append(file.name)
            print(f"  ✅ Found: {file.name}")
    
    def visit_directory(self, directory: Directory) -> None:
        pass  # Just traverse


class TreePrinter(FileSystemVisitor):
    """Print directory tree."""
    
    def __init__(self):
        self.depth = 0
    
    def visit_file(self, file: File) -> None:
        indent = "  " * self.depth
        print(f"{indent}📄 {file.name} ({file.size} bytes)")
    
    def visit_directory(self, directory: Directory) -> None:
        indent = "  " * self.depth
        print(f"{indent}📁 {directory.name}/")
        self.depth += 1
        for child in directory.children:
            child.accept(self)
        self.depth -= 1


# Demo
print("\n=== File System Visitors ===")

# Build file system
root = Directory("root")
root.add(File("readme.txt", 100))
root.add(File("config.json", 250))

docs = Directory("documents")
docs.add(File("report.pdf", 5000))
docs.add(File("notes.txt", 300))
root.add(docs)

code = Directory("code")
code.add(File("main.py", 1500))
code.add(File("utils.py", 800))
root.add(code)

print("\n1. Print tree:")
printer = TreePrinter()
root.accept(printer)

print("\n2. Calculate total size:")
size_calc = SizeCalculator()
root.accept(size_calc)
print(f"  📊 Total size: {size_calc.total_size} bytes")

print("\n3. Search for .txt files:")
searcher = FileSearcher(".txt")
root.accept(searcher)
print(f"  📋 Found {len(searcher.found)} files")

print("\n✅ Three operations without modifying File/Directory classes!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Open/Closed Principle**: Add operations without modifying classes
2. **Single Responsibility**: Separate operations from object structure
3. **Accumulate state**: Visitor can maintain state across elements
4. **Group related operations**: Keep operations together in visitor
5. **Easy to add operations**: Just create new visitor

### ❌ Disadvantages
1. **Adding elements is hard**: Must update all visitors
2. **Breaks encapsulation**: Visitor needs access to element internals
3. **Circular dependency**: Elements and visitors know each other
4. **Complexity**: Double dispatch can be confusing

## Double Dispatch Explained

Normal method call (single dispatch):
```python
shape.draw()  # Method depends on shape type only
```

Visitor (double dispatch):
```python
shape.accept(visitor)  # Depends on BOTH shape AND visitor type
# 1. shape.accept(visitor) calls visitor.visit_circle(self)
# 2. visitor.visit_circle() knows it's a Circle
```

## When to Use Visitor vs Other Patterns

**Visitor**: 
- Stable object structure
- Many operations
- Operations change frequently

**Strategy**:
- Single operation with variations
- Select algorithm at runtime

**Command**:
- Encapsulate requests
- Queue, log, undo operations

## Common Use Cases

1. **Compilers**: AST traversal for optimization, code generation
2. **File systems**: Size calculation, searching, listing
3. **Document processing**: XML/HTML parsing, transformation
4. **Graphics**: Rendering, exporting, hit testing
5. **Reports**: Multiple output formats from same data
6. **Tax calculation**: Different algorithms for different products

## Related Patterns

- **Composite**: Visitor often works with Composite structures
- **Iterator**: Traverse structure, Visitor operates on elements
- **Interpreter**: Can use Visitor to implement operations

## Best Practices

1. **Stable structure**: Only use when element classes are stable
2. **Group operations**: Put related operations in same visitor
3. **Default implementations**: Provide default visit methods
4. **Double dispatch**: Understand the mechanism
5. **Visitor interface**: One method per element type
6. **Consider alternatives**: Visitor adds complexity

## Summary

Visitor pattern enables:
- Adding operations without modifying classes
- Separating algorithms from object structure
- Grouping related operations
- Maintaining state during traversal

Perfect for: Compilers, file systems, document processing, graphics, reports.

**Key Insight**: Separate operations from object structure using double dispatch, making it easy to add new operations without modifying existing classes!